In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# connect to db
con = sqlite3.connect('data/im.db')

# Load in data:

In [3]:
# movie_basics
basics = pd.read_sql("""SELECT * FROM movie_basics """, con).drop(['start_year'], axis=1)
basics.columns = ['movie_id', 'title', 'original_title', 'runtime', 'genres']
basics['genres'] = basics['genres'].str.replace(',', ' ')

In [4]:
# directors bridge table
directors = pd.read_sql(""" SELECT * FROM directors """, con)
directors = directors.drop_duplicates()

# drop movies with multiple directors
dirs_grouped = directors.groupby('movie_id').count().sort_values('person_id', ascending=False).reset_index()
multiple_dirs = dirs_grouped.query(" person_id > 1 ")['movie_id'].values
dropped_indexes = directors[directors['movie_id'].isin(multiple_dirs)]
directors = directors.drop(dropped_indexes.index)

In [5]:
# persons table to link directors -> movie_basics
persons = pd.read_sql(""" SELECT * FROM persons """, con).drop(['birth_year', 'death_year', 'primary_profession'], axis=1)

In [6]:
# gross profit data
gross = pd.read_csv('data/bom.movie_gross.csv.gz').drop(['year'], axis=1)
gross['foreign_gross'] = gross['foreign_gross'].str.replace(',', '').astype(float)

In [7]:
# budget/profit data
budget = pd.read_csv('data/tn.movie_budgets.csv.gz').drop(['id'], axis=1)
budget['month'] = pd.to_datetime(budget['release_date']).dt.month
budget.columns = ['release_date', 'title', 'production_budget', 'domestic_gross', 'worldwide_gross', 'month']

# cast profit columns to float
for col in budget.columns[2:5]:
    budget[col] = budget[col].str.replace('[\$\,]', '', regex=True).astype(float)

In [8]:
# the movie database
tmdb = pd.read_csv('data/tmdb.movies.csv.gz', index_col='Unnamed: 0').drop(['id','original_language','original_title'], axis=1)
tmdb['genre_ids'] = tmdb['genre_ids'].str.replace('[\[\]\,]', '', regex=True)

# map genre_id to genre
def map_genres(ids_string, genre_map):
    return ' '.join([genre_map[int(genre_id)] for genre_id in ids_string.split() if int(genre_id) in genre_map])

genre_map = {28:'Action',16:'Animation',99:'Documentary',18:'Drama',10751:'Family',14:'Fantasy',36:'History',35:'Comedy',
             10752:'War',80:'Crime',10402:'Musical',9648:'Mystery',10749:'Romance',878:'Science Fiction',27:'Horror',
             10770:'TV movie',53:'Thriller',37:'Western',12:'Adventure'}

tmdb['genre_names'] = tmdb['genre_ids'].apply(lambda x: map_genres(x, genre_map))
tmdb = tmdb.drop(['genre_ids', 'release_date'], axis=1)

# Combine 6 tables:

In [9]:
# merge to get directors for each movie title
people = persons.merge(directors, on='person_id')
merge_1 = basics.merge(people, on='movie_id').drop(['movie_id', 'original_title', 'person_id', 'person_id'], axis=1)

In [10]:
# final dataframe
merge_2 = merge_1.merge(gross, on='title', how='outer').merge(
    tmdb, on='title', how='outer').merge(budget, on='title', how='outer')

In [11]:
# calculate ROI
merge_2['roi'] = (merge_2['worldwide_gross'] - merge_2['production_budget']) / merge_2['production_budget']

In [21]:
# only take movies with cost and profit data
final_df = merge_2[merge_2['roi'].notnull()]

In [22]:
# condense the two genre columns
def combine_genres(genre1, genre2):
    # ensure the genres are strings, otherwise treat them as empty strings
    genre1 = genre1 if isinstance(genre1, str) else ''
    genre2 = genre2 if isinstance(genre2, str) else ''
    
    def replace_word(s, target, replacement):
        return ' '.join([replacement if word == target else word for word in s.split()])
    
    # replace 'Music' with 'Musical'
    genre1 = replace_word(genre1, 'Music', 'Musical')
    genre2 = replace_word(genre2, 'Music', 'Musical')

    # if either 'Science' or 'Fiction' exist, replace them with 'Sci-Fi'
    if 'Science' in genre1 or 'Fiction' in genre1:
        genre1 = genre1.replace('Science', '').replace('Fiction', 'Sci-Fi')

    if 'Science' in genre2 or 'Fiction' in genre2:
        genre2 = genre2.replace('Science', '').replace('Fiction', 'Sci-Fi')
    
    combined = set(genre1.split() + genre2.split())
    return ' '.join(combined)

final = final_df.copy()
final['genre'] = final_df.apply(lambda row: combine_genres(row['genres'], row['genre_names']), axis=1)

In [23]:
# remove duplicate rows
final = final.drop_duplicates()

# remove duplicate titles
final = final[final.title.duplicated() == False]

# remove movies with no genre
final = final[final['genre'] != '']

# drop rows with no profit data
final = final.drop(final.query("worldwide_gross == 0").index)

final = final.drop(['genres','genre_names','foreign_gross','domestic_gross_x', 'domestic_gross_y'], axis=1)

In [25]:
final.to_csv('data/final.csv')